In [1]:
import os
import sys
import folium.features
import branca

# Obtenir le répertoire de travail courant
current_dir = os.getcwd()

# Obtenir le répertoire parent de `maps` (qui est `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Ajouter `src` au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
import matplotlib.pyplot as plt

from city_utils import *
from useful_methods.data_processing import extract_data
from useful_methods.neighbours_delaunay.enhanced_criteria import *
from useful_methods.ihm.maps.mapUtils import addLegend


In [3]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=',')
df_extracted = extract_data(df, provider='Orange', techno='4g')

In [4]:
labels_1 = city_detection(df_extracted[['x', 'y']])['labels']
labels_2 = city_detection_enhanced(df_extracted[['x', 'y']])['labels']

def mean_distance_to_city_level(v):
    if (v>4):
        return 4
    if (v>2):
        return 3
    if (v>1):
        return 2
    return 1

mean_distances = mean_distance_to_NN(df_extracted[['x', 'y']], n_neighbours=3)
labels_3_detailed = mean_distances.apply(mean_distance_to_city_level)
labels_3 = labels_3_detailed.apply(lambda v : 1 if(v<=2) else -1 )

## Plotting cities

In [5]:
colors = labels_1.apply(lambda v : 'green' if v==-1 else 'blue')
map1 = folium.Map(location=[46.8, 2.3], zoom_start=6, tiles="Cartodb Positron")

for ind, latitude, longitude in df_extracted[['latitude', 'longitude']].itertuples():
    popup = f'ville (DBScan, HDBScan) :\n({labels_1[ind]!=-1},{labels_2[ind]!=-1})'
    map1.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], fill_opacity=0.2, radius=1.5, color = colors[ind], popup=popup))

map1.save("../../out/maps/city-detection-DBScan.html")

In [6]:
colors = labels_2.apply(lambda v : 'green' if v==-1 else 'blue')
map2 = folium.Map(location=[46.8, 2.3], zoom_start=6, tiles="Cartodb Positron")

for ind, latitude, longitude in df_extracted[['latitude', 'longitude']].itertuples():
    popup = f'ville (DBScan, HDBScan) :\n({labels_1[ind]!=-1},{labels_2[ind]!=-1})'
    map2.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], fill_opacity=0.2, radius=1.5, color = colors[ind], popup=popup))

map2.save("../../out/maps/city-detection-HDBScan.html")

In [7]:
colors = labels_3_detailed.apply({1:'#030464',2:'#069AF3',3:'#02D4BB',4:'#0DBF75'}.get)
map3 = folium.Map(location=[46.8, 2.3], zoom_start=6, tiles="Cartodb Positron")

for ind, latitude, longitude in df_extracted[['latitude', 'longitude']].itertuples():
    popup = f'ville (DBScan, HDBScan) :\n({labels_1[ind]!=-1},{labels_2[ind]!=-1})'
    map3.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], fill_opacity=0.2, radius=1.5, color = colors[ind], popup=popup))

map3.save("../../out/maps/city-detection-KNN.html")

In [8]:
fig = branca.element.Figure()
for i, map in zip([1,2,4], [map1, map2, map3]):
    subplot = fig.add_subplot(2, 2, i)
    subplot.add_child(map)
fig.save("../../out/maps/city-detection.html")

## Comparing city detections

In [9]:
def labelsToColor(l1, l2): 
    if ((l1==-1) and (l2==-1)):
        return 'green'
    if((l1==-1) or (l2==-1)):
        return 'blue'
    return 'red'

In [10]:
colors = pd.Series(data=[labelsToColor(l1, l2) for l1, l2 in zip(labels_1, labels_2)], index=df_extracted.index)
map4 = folium.Map(location=[46.8, 2.3], zoom_start=6, tiles="Cartodb Positron")

for ind, latitude, longitude in df_extracted[['latitude', 'longitude']].itertuples():
    popup = f'ville (DBScan, HDBScan) :\n({labels_1[ind]!=-1},{labels_2[ind]!=-1})'
    map4.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], fill_opacity=0.2, radius=1.5, color = colors[ind], popup=popup))

map4.save("../../out/maps/city-detection-comparison_DBScan-HDBScan.html")

In [11]:
colors = pd.Series(data=[labelsToColor(l2, l3) for l2, l3 in zip(labels_2, labels_3)], index=df_extracted.index)
map5 = folium.Map(location=[46.8, 2.3], zoom_start=6, tiles="Cartodb Positron")

for ind, latitude, longitude in df_extracted[['latitude', 'longitude']].itertuples():
    popup = f'ville (HDBScan, KNN) :\n({labels_2[ind]!=-1},{labels_3[ind]!=-1})'
    map5.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], fill_opacity=0.2, radius=1.5, color = colors[ind], popup=popup))

map5.save("../../out/maps/city-detection-comparison_HDBScan-3NN.html")

In [12]:
colors = pd.Series(data=[labelsToColor(l1, l3) for l1, l3 in zip(labels_1, labels_3)], index=df_extracted.index)
map6 = folium.Map(location=[46.8, 2.3], zoom_start=6, tiles="Cartodb Positron")

for ind, latitude, longitude in df_extracted[['latitude', 'longitude']].itertuples():
    popup = f'ville (DBScan, KNN) :\n({labels_1[ind]!=-1},{labels_3[ind]!=-1})'
    map6.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], fill_opacity=0.2, radius=1.5, color = colors[ind], popup=popup))

map6.save("../../out/maps/city-detection-comparison_DBScan-3NN.html")

In [13]:
fig = branca.element.Figure()
for i, map in zip([1,2,4], [map4, map5, map6]):
    subplot = fig.add_subplot(2, 2, i)
    subplot.add_child(map)
addLegend(fig,{'ville-ville':'red', 'ville-campagne':'blue', 'campagne-campagne':'green'})
fig.save("../../out/maps/city-detection-comparison.html")

In [14]:
city_comparison(labels_1, labels_2)

(0.6881406044678056,
 0.028055190538764783,
 0.051938239159001316,
 0.2318659658344284)

In [15]:
city_comparison(labels_2, labels_3)

(0.6264126149802891,
 0.11366622864651774,
 0.01011826544021025,
 0.2498028909329829)

In [16]:
city_comparison(labels_1, labels_3)

(0.62973061760841,
 0.08646517739816031,
 0.006800262812089356,
 0.27700394218134033)